In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
file_path = '../data/raw/CoAID/05-01-2020/ClaimFakeCOVID-19.csv'


fake = pd.read_csv('../data/raw/CoAID/05-01-2020/ClaimFakeCOVID-19.csv')
real = pd.read_csv('../data/raw/CoAID/05-01-2020/ClaimRealCOVID-19.csv')

fake_titles = fake[['title']].copy()
fake_titles['label'] = 1  # Misinformation

real_titles = real[['title']].copy()
real_titles['label'] = 0  # Factual

dataset = pd.concat([fake_titles, real_titles], axis=0)
dataset = dataset.rename(columns={'title': 'text'})

min_samples = min(len(fake_titles), len(real_titles))
balanced = pd.concat([
    fake_titles.sample(min_samples),
    real_titles.sample(min_samples)
]).sample(frac=1).reset_index(drop=True)

train, temp = train_test_split(balanced, test_size=0.2, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

train.to_csv('../data/processed/train.csv', index=False)
val.to_csv('../data/processed/val.csv', index=False)
test.to_csv('../data/processed/test.csv', index=False)

In [8]:
print(f"Total samples: {len(balanced)}")
print(f"Class balance:\n{balanced['label'].value_counts()}")
print(f"\nSample fake claim: {balanced[balanced['label'] == 1]['title'].iloc[0]}")
print(f"Sample real claim: {balanced[balanced['label'] == 0]['title'].iloc[0]}")

Total samples: 54
Class balance:
label
1    27
0    27
Name: count, dtype: int64

Sample fake claim: "Home remedies can cure and protect against COVID-19"
Sample real claim: "What are the International Health Regulations?"


In [ ]:
import tweepy
from tqdm import tqdm

BEARER_TOKEN = '???????????????????????????????????????'

client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)

fake_df = pd.read_csv('../data/raw/CoAID/05-01-2020/ClaimFakeCOVID-19_tweets.csv', header=None)
real_df = pd.read_csv('../data/raw/CoAID/05-01-2020/ClaimRealCOVID-19_tweets.csv', header=None)

fake_ids = fake_df[1].astype(str).tolist()
real_ids = real_df[1].astype(str).tolist()

def hydrate_tweets(tweet_ids):
    tweets = []
    for i in tqdm(range(0, len(tweet_ids), 100)):
        batch = tweet_ids[i:i+100]
        try:
            batch = [str(tid).strip() for tid in batch if str(tid).strip().isdigit()]
            if not batch:
                continue
            response = client.get_tweets(ids=batch, tweet_fields=['text'])
            if response.data:
                tweets.extend([tweet.text for tweet in response.data])
        except Exception as e:
            print(f"Error fetching batch {i}: {e}")
    return tweets

fake_tweets = hydrate_tweets(fake_ids)
real_tweets = hydrate_tweets(real_ids)

df_fake = pd.DataFrame({'text': fake_tweets, 'label': 1})
df_real = pd.DataFrame({'text': real_tweets, 'label': 0})

df_all = pd.concat([df_fake, df_real]).sample(frac=1).reset_index(drop=True)

df_all.to_csv("../data/processed/combined_raw_tweets.csv", index=False)

 40%|████      | 2/5 [00:00<00:00,  6.35it/s]

Error fetching batch 0: 401 Unauthorized
Unauthorized
Error fetching batch 100: 401 Unauthorized
Unauthorized


 80%|████████  | 4/5 [00:00<00:00,  6.66it/s]

Error fetching batch 200: 401 Unauthorized
Unauthorized
Error fetching batch 300: 401 Unauthorized
Unauthorized


100%|██████████| 5/5 [00:00<00:00,  6.57it/s]


...

Error fetching batch 5900: 401 Unauthorized
Unauthorized
Error fetching batch 6000: 401 Unauthorized
Unauthorized


 98%|█████████▊| 63/64 [00:09<00:00,  7.19it/s]

Error fetching batch 6100: 401 Unauthorized
Unauthorized
Error fetching batch 6200: 401 Unauthorized
Unauthorized


100%|██████████| 64/64 [00:09<00:00,  6.97it/s]

Error fetching batch 6300: 401 Unauthorized
Unauthorized
